<a href="https://colab.research.google.com/github/Uwimbabazi-Keza/Breast-Cancer-Prediction-Model/blob/main/Transfer_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cbis-ddsm-breast-cancer-image-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1115384%2F1873742%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240521%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240521T043320Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D55f8752bd48a630e3fdb2eff6fd1b45d02d29b439d6e71764b3100a5f8ab78999cbb69e0bab44ec1d207bd82182f001e3e5e42f02a4e73d97dcdf23fde33f35923319307ba0091a6a81203a34005c99ea75e76c06a0e182a2b8cc603cf8604e123922a341893bc3e4469d2f8060f7bebaddbb1769cdaad874582a9861fe82430d971b4ad6c0e53a0a6b308e5da09106bd69b1da6983053f4757a65080637a3a4d577e02844a4c516927a908df655d3f4c4858f97c653e4c70c64fe677bf43fd402bbe80c9a4a25371772a7ccfb6d5d8297cada12e6851e3247022bb32c547d42e03e02228258cb98d315a5c29369b53077774ee5933aded8a61cfe5f04412230,mobilenet-v1/tensorflow2/100-224-classification/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F1585%2F1950%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240521%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240521T043320Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D78fd5e70ae53091e934d7e4443c40cee67335692c78c7ce59f79aee0b4fb3590dd6caeb494714a230367b8e28dc3fc1fde506ee7170904ad167928e0bdb710368a9140896259ae1f34912e8c27fa69659b69bb176c6fdc211ff05df5462a5bf64050f36e0393fb0b389029af67a1f470cde606d9df18ef5aa4e819d8f3f8e81e30d75cb43e8e800c1365ff3e6f9f44a38e2fb7b3993c56cfd6738df6b3a44dffedf1d922c20506fbc619cb96835a256703be5b15da67f0234370e51c5e3aca1bb0fc1531f220e3d0202b01e3bd0d4d1f3a890f713889f605e1fc852b1522092e4ec272a8d26a69e6bf6713f9824f2c391ddccee72555dcfd41adb47ae06200a3'

COLAB_DATA_PATH = '/content/data'
COLAB_CLASS_PATH = '/content/class'

shutil.rmtree(COLAB_DATA_PATH, ignore_errors=True)
os.makedirs(COLAB_DATA_PATH, 0o777, exist_ok=True)
os.makedirs(COLAB_CLASS_PATH, 0o777, exist_ok=True)

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(COLAB_DATA_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
                with ZipFile(tfile) as zfile:
                    zfile.extractall(destination_path)
            else:
                with tarfile.open(tfile.name) as tarfile:
                    tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

In [ ]:
import os
import tensorflow as tf
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
Dropout

keras.src.layers.regularization.dropout.Dropout

In [ ]:
csv_path = '/content/data/cbis-ddsm-breast-cancer-image-dataset/csv/meta.csv'
df_meta = pd.read_csv(csv_path)

dicom_data_path = '/content/data/cbis-ddsm-breast-cancer-image-dataset/csv/dicom_info.csv'
dicom_data = pd.read_csv(dicom_data_path)

image_dir = '/content/data/cbis-ddsm-breast-cancer-image-dataset/jpeg'
full_mammogram_images = dicom_data[dicom_data.SeriesDescription == 'full mammogram images'].image_path
cropped_images = dicom_data[dicom_data.SeriesDescription == 'cropped images'].image_path
roi_mask_images = dicom_data[dicom_data.SeriesDescription == 'ROI mask images'].image_path

full_mammogram_images = full_mammogram_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))
cropped_images = cropped_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))
roi_mask_images = roi_mask_images.apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))
full_mammogram_images.iloc[0]

full_mammogram_dict = dict()
cropped_dict = dict()
roi_mask_dict = dict()

for dicom in full_mammogram_images:
    # print(dicom)
    key = dicom.split("/")[5]
    # print(key)
    full_mammogram_dict[key] = dicom
for dicom in cropped_images:
    key = dicom.split("/")[5]
    cropped_dict[key] = dicom
for dicom in roi_mask_images:
    key = dicom.split("/")[5]
    roi_mask_dict[key] = dicom



In [ ]:
mass_train_data = pd.read_csv('/content/data/cbis-ddsm-breast-cancer-image-dataset/csv/mass_case_description_train_set.csv')
mass_test_data = pd.read_csv('/content/data/cbis-ddsm-breast-cancer-image-dataset/csv/mass_case_description_test_set.csv')
calc_train_data = pd.read_csv('/content/data/cbis-ddsm-breast-cancer-image-dataset/csv/calc_case_description_train_set.csv')
calc_test_data = pd.read_csv('/content/data/cbis-ddsm-breast-cancer-image-dataset/csv/calc_case_description_test_set.csv')

def fix_image_path_mass(dataset):
    for i, img in enumerate(dataset.values):
        img_name = img[11].split("/")[2]
        if img_name in full_mammogram_dict:
            dataset.iloc[i, 11] = full_mammogram_dict[img_name]

        img_name = img[12].split("/")[2]
        if img_name in cropped_dict:
            dataset.iloc[i, 12] = cropped_dict[img_name]

        img_name = img[13].split("/")[2]
        if img_name in roi_mask_dict:
            dataset.iloc[i, 13] = roi_mask_dict[img_name]

fix_image_path_mass(mass_test_data)
fix_image_path_mass(mass_train_data)

In [ ]:
rename_columns = {
    'left or right breast': 'left_or_right_breast',
    'image view': 'image_view',
    'abnormality id': 'abnormality_id',
    'abnormality type': 'abnormality_type',
    'mass shape': 'mass_shape',
    'mass margins': 'mass_margins',
    'image file path': 'image_file_path',
    'cropped image file path': 'cropped_image_file_path',
    'ROI mask file path': 'ROI_mask_file_path'
}

mass_train = mass_train_data.rename(columns=rename_columns)
mass_test = mass_test_data.rename(columns=rename_columns)

In [ ]:
benign=mass_train[mass_train.pathology=="BENIGN"]
benign_without_callback=mass_train[mass_train.pathology=="BENIGN_WITHOUT_CALLBACK"]
malignant=mass_train[mass_train.pathology=="MALIGNANT"]

benign_images=benign.image_file_path
malignant_images=malignant.image_file_path
benign_without_callback_images=benign_without_callback.image_file_path

benign_path='/content/data/cbis-ddsm-breast-cancer-image-dataset/class/benign'
benign_wcb_path='/content/data/cbis-ddsm-breast-cancer-image-dataset/class/benignwithoutcallback'
malignant_path='/content/data/cbis-ddsm-breast-cancer-image-dataset/class/malignant'
os.makedirs(benign_path,exist_ok=True)
os.makedirs(malignant_path,exist_ok=True)

for ben_path in benign_images:
  img_filename1=os.path.basename(ben_path)
  destination_path1=os.path.join(benign_path,img_filename1)
  shutil.copy(ben_path,destination_path1)

for mal_path in malignant_images:
  img_filename1=os.path.basename(mal_path)
  destination_path1=os.path.join(malignant_path,img_filename1)
  shutil.copy(mal_path,destination_path1)

In [ ]:
# Define image dimensions
IMG_HEIGHT, IMG_WIDTH = 512, 512
CHANNELS = 1  # Grayscale

# Define file paths for benign and malignant images (replace with your arrays)
benign_file_paths = benign_images
malignant_file_paths = malignant_images

# Create DataFrame with file paths and labels
benign_df = pd.DataFrame({'file_paths': benign_file_paths, 'labels': 'benign'})
malignant_df = pd.DataFrame({'file_paths': malignant_file_paths, 'labels': 'malignant'})

# Concatenate DataFrames
df = pd.concat([benign_df, malignant_df], ignore_index=True)

# Split data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['labels'])

In [ ]:
# Create data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='file_paths',
    y_col='labels',
    target_size=(512, 512),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    seed=42
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='file_paths',
    y_col='labels',
    target_size=(512, 512),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary',
    shuffle=False,
    seed=42
)

Found 971 validated image filenames belonging to 2 classes.
Found 243 validated image filenames belonging to 2 classes.


In [ ]:
#Build Model
def build_model(base_model):
    model = Sequential()
    model.add(Input(shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)))
    model.add(Conv2D(3, (3, 3), padding='same'))  # Converting grayscale to RGB
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Load pre-trained models
vgg16_base = VGG16(weights='imagenet', include_top=False)
resnet50_base = ResNet50(weights='imagenet', include_top=False)
inceptionv3_base = InceptionV3(weights='imagenet', include_top=False)

# Freeze the layers of the base models
for layer in vgg16_base.layers:
    layer.trainable = False
for layer in resnet50_base.layers:
    layer.trainable = False
for layer in inceptionv3_base.layers:
    layer.trainable = False

# Build models with the base models
vgg16_model = build_model(vgg16_base)
resnet50_model = build_model(resnet50_base)
inceptionv3_model = build_model(inceptionv3_base)

# Compile the models
vgg16_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])
resnet50_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])
inceptionv3_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])


In [ ]:
# Define callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

# Train VGG16 model
history_vgg16 = vgg16_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3,
    callbacks=callbacks
)

# Train ResNet50 model
history_resnet50 = resnet50_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3,
    callbacks=callbacks
)

# Train InceptionV3 model
history_inceptionv3 = inceptionv3_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3,
    callbacks=callbacks
)


Epoch 1/3
 6/31 [====>.........................] - ETA: 1:24:33 - loss: 0.7075 - accuracy: 0.5781 - precision: 0.5867 - recall: 0.8224

In [ ]:
# Evaluate models
vgg16_eval = vgg16_model.evaluate(val_generator)
resnet50_eval = resnet50_model.evaluate(val_generator)
inceptionv3_eval = inceptionv3_model.evaluate(val_generator)

# Print evaluation metrics
print("VGG16 - Accuracy: {:.2f}, Loss: {:.2f}, Precision: {:.2f}, Recall: {:.2f}".format(vgg16_eval[1], vgg16_eval[0], vgg16_eval[2], vgg16_eval[3]))
print("ResNet50 - Accuracy: {:.2f}, Loss: {:.2f}, Precision: {:.2f}, Recall: {:.2f}".format(resnet50_eval[1], resnet50_eval[0], resnet50_eval[2], resnet50_eval[3]))
print("InceptionV3 - Accuracy: {:.2f}, Loss: {:.2f}, Precision: {:.2f}, Recall: {:.2f}".format(inceptionv3_eval[1], inceptionv3_eval[0], inceptionv3_eval[2], inceptionv3_eval[3]))


In [ ]:
# Save models
vgg16_model.save('vgg16_breast_cancer_model.keras')
resnet50_model.save('resnet50_breast_cancer_model.keras')
inceptionv3_model.save('inceptionv3_breast_cancer_model.keras')
